In [1]:
# 불러오기
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
import requests
import os
import json
import numpy as np
import requests
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import re
import time
import datetime

In [2]:
# 이미지 다운로드하는 함수
def download_image(url, save_path):
    response = requests.get(url)
    response.raise_for_status()
    with open(save_path, 'wb') as file:
        file.write(response.content)

In [4]:
# page랑 brand 같이 url에 주면 그 페이지 review 크롤링
def get_review(url):
    # 제품 url 가져오기
    # BeautifulSoup
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,"html.parser")
    review = []
    review_count = len(soup.find_all("div",{"class":"review-list"}))
    for i in range(review_count):
        try:
            # goods_id
            url = soup.find_all("div",{"class":"review-list"})[i].find_all('a',{"class":"review-goods-information__link"})[0].get('href')
            match = re.search(r'goods/(\d+)/', url)
            if match:
                goods_id = match.group(1)

            # user_id
            user_id = soup.find_all("div",{"class":"review-list"})[i].find_all("p")[0].text

            # date
            date = soup.find_all("div",{"class":"review-list"})[i].find_all("p")[1].text

            # Size 
            size = re.sub(r'\s+', '', soup.find_all("div",{"class":"review-list"})[i].find_all("span",{"class":"review-goods-information__option"})[0].text)

            # text
            review_text = soup.find_all("div",{"class":"review-list"})[i].find_all("div",{"class":"review-contents__text"})[0].text

            # url
            goods_url = soup.find_all("div",{"class":"review-list"})[i].find_all('a')[2].get('href')
            #brand
            goods_brand = soup.find_all("div",{"class":"review-list"})[i].find_all('a')[3].text
            # goods name
            goods_name = soup.find_all("div",{"class":"review-list"})[i].find_all('a')[4].text

            # 도움돼요 /스타일 좋아요 / 평가
            help_eval = soup.find_all("div",{"class":"review-list"})[i].find_all('li',{"class":"review-evaluation-button__list"})[0].find('span').text
            style_eval = soup.find_all("div",{"class":"review-list"})[i].find_all('li',{"class":"review-evaluation-button__list"})[1].find('span').text
            eval_list = [item.get_text(strip=True) for item in soup.find_all("div",{"class":"review-list"})[i].find_all('li',{"class":"review-evaluation__item"})]
            rating_str = re.findall(r'\d+', soup.find_all("div",{"class":"review-list"})[i].find_all('span',{"class":"review-list__rating__active"})[0].get("style"))[0]
            rating = int(rating_str) / 20

            # review_no
            review_no = soup.find_all("div",{"class":"review-list"})[i].find_all("div",{"class":"review-contents"})[0].get("data-review-no")

            # 사진 크롤링
            img_count = len(soup.find_all("div",{"class":"review-list"})[i].find_all('img'))

            # 사진 카운트
            img_num = 0
            # 후기 사진만 저장하기
            for j in range(img_count):
                url = soup.find_all("div",{"class":"review-list"})[i].find_all('img')[j].get("src")
                if ".view" in url:
                    file_url = 'https:' + re.sub(r'\.view$', '', url)
                    # 사진 저장 경로
                    save_path = r"C:\Users\NT550009\Desktop\image\\"+f"{goods_id}_{review_no}_{img_num}.jpg"

                    # 사진 다운로드 함수
                    download_image(file_url, save_path)
                    img_num += 1
#                     print(goods_id,review_no,j,img_count,img_num,file_url)

            # user 정보 ('여성 ', ' 160cm ', ' 52kg') / 등록되어 있지 않은 경우도 있음
            if soup.find_all("div",{"class":"review-list"})[i].find_all("p",{"class":"review-profile__body_information"}) != []:
                user_sex, user_height, user_weight = soup.find_all("div",{"class":"review-list"})[i].find_all("p",{"class":"review-profile__body_information"})[0].text.split("·")
                review.append([goods_id,user_id,date,size,review_text,goods_url,goods_brand,goods_name,help_eval,style_eval,eval_list,rating,review_no,user_sex, user_height, user_weight])
            else:
                review.append([goods_id,user_id,date,size,review_text,goods_url,goods_brand,goods_name,help_eval,style_eval,eval_list,rating,review_no])
        except:
            pass
    return review

In [5]:
path = r"C:\Users\NT550009\Desktop\chromedriver_win32\chromedriver.exe" # 크롬드라이버 경로 입력
s = Service(path)
chrome_options = Options()
chrome_options.add_argument('--headless')  # 화면에 브라우저가 나타나지 않도록 headless 모드로 실행합니다.

driver = webdriver.Chrome(service=s)



In [6]:

# 브랜드 리스트 가져오기 (자신이 맡은 브랜드 리스트 형성하기)
brand = pd.read_excel("brand_name.xlsx")
brand_list = [text.replace(' ', '').lower() for text in list(brand[0])]
print(brand_list)

['musinsastandard', 'matinkim', 'adidas', 'suare', 'yale', 'glw', 'dimitriblack', 'lafudgestore', 'toffee', 'travel', 'covernat', 'drawfit', 'satur', 'avandress', 'thenorthface', 'xero', 'spao', 'codegraphy', 'poloralphlauren', 'newbalance', 'calvinkleinjeans', 'nationalgeographic', 'espionage', 'anoblir', 'outstanding', 'trillion', 'umbro', 'lee', 'they', 'marithefrancoisgirbaud', 'thisisneverthat', 'fp142', 'partimento', 'physicaleducationdepartment', 'jemut', 'mlb', 'sculptor', 'nodearchive', 'mardimercredi', 'branded', 'brownbreath', 'perstep', 'not4nerd', 'minav', 'uniformbridge', 'nomanual', 'whatitisnt', 'intempomood', 'soverment', 'visionstreetwear', 'discoveryexpedition', 'niceghostclub', 'anderssonbell', 'xtonz', 'fottners', 'maisonmined', 'takeasy', 'cpgnstudio', 'signature', 'giordano', 'asics', 'coor', 'fila', 'pottery', 'partimentowomen', 'kodak', 'gakkaiunions', 'brenson', 'juanhomme', 'filluminate', 'kiimuir', 'modnine', 'retrieverclub', 'vivastudio', 'jeep', 'thomasmor

In [7]:
brand_list_jc = []
for idx, val in enumerate(brand_list):
    if idx % 6 ==4:
        brand_list_jc.append(val)

In [8]:
normalized_list = []

for brand in brand_list_jc:
    normalized_brand = re.sub(r'[^\w\s]', '', brand)
    normalized_list.append(normalized_brand)


In [9]:
print(normalized_list)

['yale', 'covernat', 'spao', 'espionage', 'they', 'jemut', 'brownbreath', 'whatitisnt', 'anderssonbell', 'signature', 'partimentowomen', 'kiimuir', 'yeseyesee', 'slowacid', 'ourscope', '8seconds', 'ebbetsfield', 'bottegaveneta', 'blankroom', 'bigunion', 'koleat', 'champion', 'amesworldwide', 'costumeoclock', 'musinsastandardsports', 'reebok', 'grove', 'saintlaurent', 'mozmoz', 'wkndrs', 'outdoorproducts', 'areuban', 'malbongolf', 'cavish', 'guernseywoollens', 'lecoq', 'rvd', 'nonblank', 'istkunst', 'ruggedhouse', 'sortie', 'burberry', 'rolarola', 'awesomestudio', 'flareup', 'tommyhilfiger', 'ogarp', 'ourdayz', 'publicicon', 'grailz', 'sinoon', 'lengagement', 'sk8errock', 'painorpleasure', 'snowpeak', 'aott', 'nastykick', 'mizuno', 'saintjames', 'noun', 'hideout', 'maxmara', 'doubleflag', 'videojockey', 'nfl', 'esn', 'slickandeasy', 'claco', 'pepperseasoning', 'worthwhilemovement', 'arcteryx', 'anotherarchive', 'grish', 'descente', 'expiredgirl', 'marni', 'basichouse', 'wrangler', '2000

# 중간 연결 끊김 및 브랜드별 페이지 수 10개로 조정
마지막 : espionage의 546 page 중에 154 page까지 완료 / 4094

In [10]:
normalized_list = normalized_list[4:]

In [11]:
# 전제 review 저장
review = []
count = 80000

for brand in normalized_list:
    url = f'https://www.musinsa.com/goods/reviews/lists?type=style&searchYear=2022&searchMonth=&searchDay=&maxRt=2022&minRt=2009&brand={brand}&page=1&sort=new'
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,"html.parser")
    tot_page = 10
    for page in range(1,tot_page+1):
        url = f'https://www.musinsa.com/goods/reviews/lists?type=style&searchYear=2022&searchMonth=&searchDay=&maxRt=2022&minRt=2009&brand={brand}&page={page}&sort=comment_new_desc'
        review = review + get_review(url)
        count += 1
        print("{0}의 {1} page 중에 {2} page까지 완료 / {3}".format(brand,tot_page,page,count))
        if count % 100 == 0:
            review_df = pd.DataFrame(review,columns = ["goods_id","user_id","date","size","text","goods_url","goods_brand","goods_name","help_eval","style_eval","eval_list","rating","review_no","user_sex", "user_height", "user_weight"])
            review_df.to_csv(f"review/review_{count*20}.csv")
            print(f"review 저장완료_{count*20}")

they의 10 page 중에 1 page까지 완료 / 80001
they의 10 page 중에 2 page까지 완료 / 80002
they의 10 page 중에 3 page까지 완료 / 80003
they의 10 page 중에 4 page까지 완료 / 80004
they의 10 page 중에 5 page까지 완료 / 80005
they의 10 page 중에 6 page까지 완료 / 80006
they의 10 page 중에 7 page까지 완료 / 80007
they의 10 page 중에 8 page까지 완료 / 80008
they의 10 page 중에 9 page까지 완료 / 80009
they의 10 page 중에 10 page까지 완료 / 80010
jemut의 10 page 중에 1 page까지 완료 / 80011
jemut의 10 page 중에 2 page까지 완료 / 80012
jemut의 10 page 중에 3 page까지 완료 / 80013
jemut의 10 page 중에 4 page까지 완료 / 80014
jemut의 10 page 중에 5 page까지 완료 / 80015
jemut의 10 page 중에 6 page까지 완료 / 80016
jemut의 10 page 중에 7 page까지 완료 / 80017
jemut의 10 page 중에 8 page까지 완료 / 80018
jemut의 10 page 중에 9 page까지 완료 / 80019
jemut의 10 page 중에 10 page까지 완료 / 80020
brownbreath의 10 page 중에 1 page까지 완료 / 80021
brownbreath의 10 page 중에 2 page까지 완료 / 80022
brownbreath의 10 page 중에 3 page까지 완료 / 80023
brownbreath의 10 page 중에 4 page까지 완료 / 80024
brownbreath의 10 page 중에 5 page까지 완료 / 80025
brownbreath의 10 page 중에 6 pa